In [1]:
# importing all the required modules

from importlib import reload
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import os
import errno
import string
import collections
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from prettytable import PrettyTable
import time
from datetime import timedelta
from sklearn.metrics import recall_score,precision_score,average_precision_score,f1_score
from sklearn.metrics import accuracy_score,roc_auc_score,mean_absolute_error,cohen_kappa_score
from sklearn.preprocessing import label_binarize,LabelEncoder
from imblearn.metrics import geometric_mean_score
from sklearn import metrics
from sklearn.metrics.cluster import homogeneity_score,completeness_score
import statistics
import math
import sklearn.metrics 
from sklearn.model_selection import KFold,StratifiedKFold
import csv
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from scipy.spatial import distance
import statistics
from scipy.stats import pearsonr,entropy,mode
from scipy.io import arff
from sklearn.metrics import pairwise_distances
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import load_iris
import warnings
import random
from statistics import mode
random.seed(0)
warnings.filterwarnings('ignore')

def loadNonIRDatasets(dataset):
    documents,labels=[],[]
    fname=dataset+'.csv'
    with open(fname, 'r') as file:
        reader = csv.reader(file, delimiter = ',')
        for row in reader:
            if len(row)>0:
                if '?' in row:
                    continue
                doc=row[0:len(row)-2]
                try:
                    doc=[abs(float(d)) for d in doc]
                except:
                    continue
                documents.append(doc)
                #print(row)
                labels.append(row[len(row)-1])

    return documents,labels

def Cosine(a, b):#distance
    return distance.cosine(a,b)

def Euclidean(a, b):#distance
    return distance.euclidean(a,b)

def PrintDetails(metric="smtp",measure= "Accuracy",Arr=None,kList=[5,15,30,45],train_time=0,test_time=0,std=[]):
    x = PrettyTable()
    #kList=[1,3,5,9,15,30,45]#,70,90,120]
    if std==[]:
        x.field_names = ["k",measure,"Train Time","Test time","Total Time"]
    else:
        x.field_names = ["Split",measure,"std","Train Time","Test time","Total Time"]
        
    print("metric",metric)
    print("measure",measure)
    tables.write(("metric\t"+str(metric)+"\n").encode())
    tables.write(("measure\t"+str(measure)+"\n").encode())
    average=0
    total_time=np.add(train_time,test_time)
    for i in range(0,len(Arr)):
        print(train_time[i])
        tr_time=str(timedelta(seconds=(train_time[i])))
        ts_time=str(timedelta(seconds=(test_time[i])))
        
        tot_time=str(timedelta(seconds=(total_time[i])))
        if std==[]:
            x.add_row([str(kList[i]),round(Arr[i],4),tr_time,ts_time,tot_time]) 
        else:
            x.add_row([str(kList[i]),round(Arr[i],4),round(std[i],4),tr_time,ts_time,tot_time]) 
            
        average+=Arr[i]
    if std==[]:
        x.add_row(["Average",round((average/len(Arr)),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),np.std(train_time),
                   np.std(test_time),np.std(total_time)])   
    else:
        x.add_row(["Average",round((average/len(Arr)),4),round(np.mean(std),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),round(np.std(std),4),
                   round(np.std(train_time),10),
                   round(np.std(test_time),10),
                   round(np.std(total_time),10)])
        
    x.padding_width =1
    tables.write(str(x).encode())
    print(x)

def findDI(train_data,train_labels,metric,k):
    clss=set(train_labels)
    func=globals()[metric]
    radius={}
    th=0.5
    for cl in clss:
        indices=[i for i in range(0,len(train_labels)) if train_labels[i]==cl]
        elements=train_data[indices,:]
        distMat=pairwise_distances(elements,elements,metric=func)
        radius[cl]=np.mean(distMat)
    weights=[]
   
    distances=pairwise_distances(train_data,train_data,metric=func)
    for i in range(0,train_data.shape[0]):
        r=radius[train_labels[i]]
        ID=0
        val=0
        c=1
        for nn,neigh in enumerate(train_labels):
            if distances[i][nn]<r and i!=nn:
                c=c+1
                if neigh==train_labels[i]:
                    val=val+1
        ID=val/c
        ##############find weights using validity ###############
        #w=np.sum(val*(1/(distances[i]+alpha)))
        weights.append(ID)
    return weights
    

        
def classify(k=3,metric="euclidean",termOccurance=None, docOccurance=None):
    accuracy=[]
    precision=[]
    Recall=[]
    fMeasure=[]
    mprecision=[]
    mRecall=[]
    mfMeasure=[]
    roc=[]
    y_pred=[]
    mae=[]
    kappa=[]
    averageMeanPrecison=[]
    func=globals()[metric]
    kList=[5,15,30,45]
    var=np.zeros(train_data.shape[1])
    train_time=[]
    test_time=[]
    for k in kList:
        time1=time.time()
        print("k",k)
        weights=findDI(train_data,train_labels,metric,k)
        classifier = NearestNeighbors(n_neighbors=k,metric=func)
        if k>len(train_labels):
             classifier = NearestNeighbors(n_neighbors=len(train_labels),metric=func)
        classifier.fit(train_data,train_labels)
        distances,neighVals=classifier.kneighbors(test_data,return_distance=True)
        pred=[]
        time2=time.time()
        for i in range(0,test_data.shape[0]):
            neighs=np.array(neighVals[i,:],dtype=int)
            #print(neighs)
            neighLabels = [train_labels[i] for i in neighs]
            dLabels=set(neighLabels)
            weightSum=0
            maxweight=0
            predLabel=''
            for lbl in dLabels:
                weightSum=0
                #print(neighs)
                weightSum=np.mean([weights[ii]/distances[i][j] for j,ii in enumerate(neighs) if train_labels[ii]==lbl])
                #print(weightSum)
                if weightSum>=maxweight:
                    maxweight=weightSum
                    predLabel=lbl
            pred.append(predLabel)  
        y_pred.append(pred)
        time3=time.time()
        train_time.append(time2-time1)
        test_time.append(time3-time2)
 
   
    class_dict={}
    classes=list(set(labels))
    for i in range(0,len(classes)):
        class_dict[classes[i]]=i
    print(class_dict)
    num_testLabels=[]
    for i in range(0,len(test_labels)):
        num_testLabels.append(class_dict[test_labels[i]])
       
    for i in range(0,len(y_pred)):
        test_labels1=list(test_labels)
        #Accuracy
       
        accuracy.append(accuracy_score(test_labels, y_pred[i]))
        
        #Precison
        precision.append(precision_score(test_labels, y_pred[i],average='macro'))
         
        #Recall
        Recall.append(recall_score(test_labels, y_pred[i],average='macro'))
           
        #f measure
        fMeasure.append(f1_score(test_labels, y_pred[i],average='macro'))
        
        #Precison
        mprecision.append(precision_score(test_labels, y_pred[i],average='micro'))
         
        #Recall
        mRecall.append(recall_score(test_labels, y_pred[i],average='micro'))
           
        #f measure
        mfMeasure.append(f1_score(test_labels, y_pred[i],average='micro'))
        
        kappa.append(cohen_kappa_score(test_labels, y_pred[i]))
      
        test = label_binarize(test_labels1, classes=categories).reshape((-1))
        pred = label_binarize( y_pred[i], classes=categories).reshape((-1))
        
        #roc
        try:
            roc.append(roc_auc_score(test,pred))
        except:
            roc.append(-1)
            
        #Average Mean Precision
        #y_val_true, val_pred = y_val_true.reshape((-1)), val_pred.reshape((-1))
        averageMeanPrecison.append(average_precision_score(test, pred))
        #MAE
        num_pred=[]
        for j in range(0,len(y_pred[i])):
            num_pred.append(class_dict[y_pred[i][j]])
        mae.append(mean_absolute_error(num_testLabels, num_pred))
        
        #PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="Average Mean Precison",Arr=averageMeanPrecison)


    
   #Accuracy
    PrintDetails(metric=metric,measure="Accuracy",Arr=accuracy,train_time=train_time,test_time=test_time)
    
    #Precison
    PrintDetails(metric=metric,measure="Precision",Arr=precision,train_time=train_time,test_time=test_time)
    #Precison
    PrintDetails(metric=metric,measure="micro Precision",Arr=mprecision,train_time=train_time,test_time=test_time)
    #Recall
    PrintDetails(metric=metric,measure="Recall",Arr=Recall,train_time=train_time,test_time=test_time)
     #Recall
    PrintDetails(metric=metric,measure="micro Recall",Arr=mRecall,train_time=train_time,test_time=test_time)
    #f measure
    PrintDetails(metric=metric,measure="F Measure",Arr=fMeasure,train_time=train_time,test_time=test_time)
    
    #f measure
    PrintDetails(metric=metric,measure="micro F Measure",Arr=mfMeasure,train_time=train_time,test_time=test_time)
    
    #roc
    PrintDetails(metric=metric,measure="ROC",Arr=roc,train_time=train_time,test_time=test_time)
    #Average Mean Precision
    PrintDetails(metric=metric,measure="Average Mean Precison",Arr=averageMeanPrecison,train_time=train_time,test_time=test_time)
    #MAE
    PrintDetails(metric=metric,measure="MAE",Arr=mae,train_time=train_time,test_time=test_time)
  
    accuracy_avg.append(accuracy)
    precision_avg.append(precision)
    recall_avg.append(Recall)
    macroFMeasure_avg.append(fMeasure)
    mprecision_avg.append(mprecision)
    mrecall_avg.append(mRecall)
    mFMeasure_avg.append(mfMeasure)
    roc_avg.append(roc)
    AMP_avg.append(averageMeanPrecison)
    mae_avg.append(mae)
    avg_test_time.append(test_time)
    avg_train_time.append(train_time)
    
import glob
datasets=[]
for file_name in glob.glob("Latest Datasets\\"+'*.csv'):
    file=os.path.split(file_name)[1]
    datasets.append(os.path.splitext(file)[0])

    
for dataset in datasets:
    fname="Latest Datasets\\"+dataset
    print(fname.encode())
    arr,labels=loadNonIRDatasets(fname)
    arr=np.array(arr)
    labels=np.array(labels)
    labels=list(labels)
    le = LabelEncoder()
    labels=le.fit_transform(labels)
    categories=list(set(labels))
    measures=["Euclidean"]
    global w
    w=0
    global Weights
    for met in measures:
        n_split=1
        labels=np.array(labels)
        path = "DKNN Original Corrected\\April Task1\\Latest Datasets"
        if not os.path.exists(path):
            os.makedirs(path)
        fname=path+'\\table_'+dataset+'_'+met+'_DKNN_original_corrected.txt'
        tables = open(fname, 'wb')
        k_splits=10
        accuracy_avg=[]
        precision_avg=[]
        recall_avg=[]
        macroFMeasure_avg=[]
        mprecision_avg=[]
        mrecall_avg=[]
        mFMeasure_avg=[]
        roc_avg=[]
        AMP_avg=[]
        mae_avg=[]
        avg_test_time=[]
        avg_train_time=[]
        kf = StratifiedKFold(n_splits=k_splits)
        for trn_ind, tst_ind in kf.split(arr,labels):
            xTrain, xTest = arr[trn_ind], arr[tst_ind]
            yTrain, yTest = labels[trn_ind], labels[tst_ind]
            train_data=np.array(xTrain)
            test_data=np.array(xTest)
            train_labels=np.array(yTrain)
            test_labels=np.array(yTest)
            allData=arr
            termOcc=[]
            docOcc=[]
            tables.write("\n*************".encode())
            tables.write((" Split\t"+str(n_split)).encode())
            tables.write(" ***********\n".encode())
            n_split=n_split+1
            k=1
            print("###############")
            Weights=np.ones(train_data.shape[0])
            classify(k=k,metric=met,termOccurance=termOcc,docOccurance=docOcc)
        list1=[5,15,30,45]
        tables.write("\n************* Average Results ***********\n".encode())
        acc_std=np.std(np.array(accuracy_avg),axis=0)
        accuracy_avg=np.mean(np.array(accuracy_avg),axis=0)
        
        pr_std=np.std(np.array(precision_avg),axis=0)
        precision_avg=np.mean(np.array(precision_avg),axis=0)
        
        mpr_std=np.std(np.array(mprecision_avg),axis=0)
        mprecision_avg=np.mean(np.array(mprecision_avg),axis=0)
        
        rcl_std=np.std(np.array(recall_avg),axis=0)
        recall_avg=np.mean(np.array(recall_avg),axis=0)
        
        mrcl_std=np.std(np.array(mrecall_avg),axis=0)
        mrecall_avg=np.mean(np.array(mrecall_avg),axis=0)
        
        F_std=np.std(np.array(macroFMeasure_avg),axis=0)
        macroFMeasure_avg=np.mean(np.array(macroFMeasure_avg),axis=0)
        
        mF_std=np.std(np.array(mFMeasure_avg),axis=0)
        mFMeasure_avg=np.mean(np.array(mFMeasure_avg),axis=0)
        
        roc_std=np.std(np.array(roc_avg),axis=0)
        roc_avg=np.mean(np.array(roc_avg),axis=0)
        
        amp_std=np.std(np.array(AMP_avg),axis=0)
        AMP_avg=np.mean(np.array(AMP_avg),axis=0)
        
        mae_std=np.std(np.array(mae_avg),axis=0)
        mae_avg=np.mean(np.array(mae_avg),axis=0)
        
        avg_train_time=np.mean(np.array(avg_train_time),axis=0)
        avg_test_time=np.mean(np.array(avg_test_time),axis=0)
        #Accuracy
        PrintDetails(metric=met,measure="Accuracy",Arr=accuracy_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=acc_std)
        
        #Precision
        PrintDetails(metric=met,measure="Precision",Arr=precision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=pr_std)
        
        #Precision
        PrintDetails(metric=met,measure="Micro Precision",Arr=mprecision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mpr_std)


        #Recall
        PrintDetails(metric=met,measure="Recall",Arr=recall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=rcl_std)
        
         #Recall
        PrintDetails(metric=met,measure="Micro Recall",Arr=mrecall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mrcl_std)


        #f measure
        PrintDetails(metric=met,measure="F Measure",Arr=macroFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=F_std)
        
       
       
        #f measure
        PrintDetails(metric=met,measure="Micro F Measure",Arr=mFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mF_std)


        #roc
        PrintDetails(metric=met,measure="ROC",Arr=roc_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=roc_std)

        #average Mean precision
        PrintDetails(metric=met,measure="AMP",Arr=AMP_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=amp_std)
         
        #MAE
        PrintDetails(metric=met,measure="MAE",Arr=mae_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mae_std)
        
        tables.close()
        print("###############")





b'Latest Datasets\\banknote'
###############
k 5
k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
10.109343767166138
9.910694122314453
10.113101243972778
10.461544036865234
+----------+----------+---------------------+-----------------------+--------------------+
|    k     | Accuracy |      Train Time     |       Test time       |     Total Time     |
+----------+----------+---------------------+-----------------------+--------------------+
|    5     |  0.9203  |    0:00:10.109344   |     0:00:00.002999    |   0:00:10.112343   |
|    15    |  0.8913  |    0:00:09.910694   |     0:00:00.004987    |   0:00:09.915681   |
|    30    |  0.8986  |    0:00:10.113101   |     0:00:00.006979    |   0:00:10.120080   |
|    45    |  0.8551  |    0:00:10.461544   |     0:00:00.009973    |   0:00:10.471517   |
| Average  |  0.8913  |    0:00:10.148671   |     0:00:00.006234    |   0:00:10.154905   |
| Std Dev. |  0.0235  | 0.19832698397864879 | 0.0025767642558339123 | 0.20030968542317

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
10.055542707443237
10.051406621932983
10.182913064956665
10.427766799926758
+----------+----------+---------------------+-----------------------+---------------------+
|    k     | Accuracy |      Train Time     |       Test time       |      Total Time     |
+----------+----------+---------------------+-----------------------+---------------------+
|    5     |  0.942   |    0:00:10.055543   |     0:00:00.003992    |    0:00:10.059535   |
|    15    |  0.9275  |    0:00:10.051407   |     0:00:00.003989    |    0:00:10.055396   |
|    30    |  0.9058  |    0:00:10.182913   |     0:00:00.007979    |    0:00:10.190892   |
|    45    |  0.8623  |    0:00:10.427767   |     0:00:00.009974    |    0:00:10.437740   |
| Average  |  0.9094  |    0:00:10.179407   |     0:00:00.006483    |    0:00:10.185891   |
| Std Dev. |  0.0301  | 0.15282452557317938 | 0.0025906154823364435 | 0.15528036695975617 |
+----------+----------+-----------

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
10.304726123809814
9.973865985870361
10.210463762283325
10.21159052848816
+----------+----------+---------------------+-----------------------+---------------------+
|    k     | Accuracy |      Train Time     |       Test time       |      Total Time     |
+----------+----------+---------------------+-----------------------+---------------------+
|    5     |  0.9562  |    0:00:10.304726   |     0:00:00.002991    |    0:00:10.307717   |
|    15    |  0.9051  |    0:00:09.973866   |     0:00:00.004987    |    0:00:09.978853   |
|    30    |  0.8467  |    0:00:10.210464   |     0:00:00.007979    |    0:00:10.218443   |
|    45    |  0.8467  |    0:00:10.211591   |     0:00:00.011967    |    0:00:10.223558   |
| Average  |  0.8887  |    0:00:10.175162   |     0:00:00.006981    |    0:00:10.182143   |
| Std Dev. |  0.0457  | 0.12235215526357311 | 0.0033820431493766677 | 0.12260567091640283 |
+----------+----------+-------------

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
9.964112281799316
10.003178834915161
10.26884937286377
10.129251480102539
+----------+----------+---------------------+---------------------+---------------------+
|    k     | Accuracy |      Train Time     |      Test time      |      Total Time     |
+----------+----------+---------------------+---------------------+---------------------+
|    5     |  0.9343  |    0:00:09.964112   |    0:00:00.002991   |    0:00:09.967103   |
|    15    |  0.8978  |    0:00:10.003179   |    0:00:00.004986   |    0:00:10.008165   |
|    30    |  0.8978  |    0:00:10.268849   |    0:00:00.007979   |    0:00:10.276829   |
|    45    |  0.9051  |    0:00:10.129251   |    0:00:00.008976   |    0:00:10.138227   |
| Average  |  0.9088  |    0:00:10.091348   |    0:00:00.006233   |    0:00:10.097581   |
| Std Dev. |  0.015   | 0.11927504016998655 | 0.00237889551920192 | 0.12124499301410434 |
+----------+----------+---------------------+---------

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
10.11313509941101
9.984237670898438
10.397674560546875
10.547832012176514
+----------+----------+--------------------+-----------------------+---------------------+
|    k     | Accuracy |     Train Time     |       Test time       |      Total Time     |
+----------+----------+--------------------+-----------------------+---------------------+
|    5     |  0.9197  |   0:00:10.113135   |     0:00:00.002992    |    0:00:10.116127   |
|    15    |  0.8686  |   0:00:09.984238   |     0:00:00.004987    |    0:00:09.989225   |
|    30    |  0.8467  |   0:00:10.397675   |     0:00:00.007979    |    0:00:10.405653   |
|    45    |  0.8248  |   0:00:10.547832   |     0:00:00.008976    |    0:00:10.556808   |
| Average  |  0.865   |   0:00:10.260720   |     0:00:00.006234    |    0:00:10.266953   |
| Std Dev. |  0.0352  | 0.2232787721536761 | 0.0023783769027429526 | 0.22535259623250753 |
+----------+----------+--------------------+-

...

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.18350863456726074
0.18151378631591797
0.1825113296508789
0.18450617790222168
+----------+----------+----------------------+----------------------+-----------------------+
|    k     | Accuracy |      Train Time      |      Test time       |       Total Time      |
+----------+----------+----------------------+----------------------+-----------------------+
|    5     |   0.7    |    0:00:00.183509    |    0:00:00.000998    |     0:00:00.184506    |
|    15    |   0.7    |    0:00:00.181514    |    0:00:00.000998    |     0:00:00.182512    |
|    30    |   0.7    |    0:00:00.182511    |    0:00:00.000998    |     0:00:00.183509    |
|    45    |   0.75   |    0:00:00.184506    |    0:00:00.000998    |     0:00:00.185504    |
| Average  |  0.7125  |    0:00:00.183010    |    0:00:00.000998    |     0:00:00.184008    |
| Std Dev. |  0.0217  | 0.001115180730981565 | 1.97686242482388e-07 | 0.0011150474541303794 |
+----------+-

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.18350863456726074
0.18450665473937988
0.18054580688476562
0.176527738571167
+----------+----------+-----------------------+-----------------------+----------------------+
|    k     | Accuracy |       Train Time      |       Test time       |      Total Time      |
+----------+----------+-----------------------+-----------------------+----------------------+
|    5     |  0.6316  |     0:00:00.183509    |        0:00:00        |    0:00:00.183509    |
|    15    |  0.5263  |     0:00:00.184507    |        0:00:00        |    0:00:00.184507    |
|    30    |  0.5789  |     0:00:00.180546    |     0:00:00.000999    |    0:00:00.181545    |
|    45    |  0.5789  |     0:00:00.176528    |     0:00:00.001001    |    0:00:00.177529    |
| Average  |  0.5789  |     0:00:00.181272    |     0:00:00.000500    |    0:00:00.181772    |
| Std Dev. |  0.0372  | 0.0031024546024872655 | 0.0005001429223768714 | 0.002671605335016378 |
+----

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.1904585361480713
0.18450617790222168
0.17703604698181152
0.176527738571167
+----------+----------+----------------------+------------------------+----------------------+
|    k     | Accuracy |      Train Time      |       Test time        |      Total Time      |
+----------+----------+----------------------+------------------------+----------------------+
|    5     |  0.5263  |    0:00:00.190459    |     0:00:00.000998     |    0:00:00.191456    |
|    15    |  0.5789  |    0:00:00.184506    |     0:00:00.000998     |    0:00:00.185504    |
|    30    |  0.7895  |    0:00:00.177036    |     0:00:00.000998     |    0:00:00.178034    |
|    45    |  0.7895  |    0:00:00.176528    |     0:00:00.000997     |    0:00:00.177525    |
| Average  |  0.6711  |    0:00:00.182132    |     0:00:00.000998     |    0:00:00.183130    |
| Std Dev. |  0.1199  | 0.005752051986039946 | 1.6858739404357614e-07 | 0.005752134663115267 |
+-----

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.18154692649841309
0.18579888343811035
0.18529939651489258
0.18254303932189941
+----------+----------+-----------------------+-----------------------+----------------------+
|    k     | Accuracy |       Train Time      |       Test time       |      Total Time      |
+----------+----------+-----------------------+-----------------------+----------------------+
|    5     |  0.7895  |     0:00:00.181547    |        0:00:00        |    0:00:00.181547    |
|    15    |  0.7895  |     0:00:00.185799    |        0:00:00        |    0:00:00.185799    |
|    30    |  0.7895  |     0:00:00.185299    |     0:00:00.000998    |    0:00:00.186297    |
|    45    |  0.7895  |     0:00:00.182543    |     0:00:00.001000    |    0:00:00.183543    |
| Average  |  0.7895  |     0:00:00.183797    |     0:00:00.000499    |    0:00:00.184296    |
| Std Dev. |   0.0    | 0.0017958271574268245 | 0.0004994278947722393 | 0.001896565065337488 |
+--

+----------+--------+--------+----------------+----------------+----------------+
|  Split   |  AMP   |  std   |   Train Time   |   Test time    |   Total Time   |
+----------+--------+--------+----------------+----------------+----------------+
|    5     | 0.2379 | 0.0185 | 0:00:00.183799 | 0:00:00.000500 | 0:00:00.184299 |
|    15    | 0.2365 | 0.0211 | 0:00:00.183435 | 0:00:00.000498 | 0:00:00.183933 |
|    30    | 0.2371 |  0.02  | 0:00:00.181571 | 0:00:00.000898 | 0:00:00.182469 |
|    45    | 0.2371 |  0.02  | 0:00:00.181548 | 0:00:00.001198 | 0:00:00.182746 |
| Average  | 0.2371 | 0.0199 | 0:00:00.182588 | 0:00:00.000774 | 0:00:00.183362 |
| Std Dev. | 0.0005 | 0.0009 |  0.0010366214  |  0.0002941691  |  0.0007713564  |
+----------+--------+--------+----------------+----------------+----------------+
metric Euclidean
measure MAE
0.18379898071289064
0.1834346294403076
0.1815709352493286
0.18154845237731934
+----------+--------+--------+----------------+----------------+---------

In [2]:
"test"

'test'